<a href="https://colab.research.google.com/github/41371116h/PL-Repo./blob/main/%E7%A4%BE%E5%9C%98%E7%B8%BD%E5%8B%99%E8%A8%98%E5%B8%B3%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gradio as gr
import pandas as pd
import datetime
import gspread
from google.colab import auth
from google.auth import default

# ===== Google Sheets 設定 =====
SHEET_URL = "https://docs.google.com/spreadsheets/d/1GScHTHISiioV89XO5twgGl-IpaYG-0nMwhLRTizSpNI/edit?pli=1&gid=0#gid=0"
WORKSHEET_NAME = "工作表1"

# ===== Google 授權 =====
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
gsheets = gc.open_by_url(SHEET_URL)
worksheet = gsheets.worksheet(WORKSHEET_NAME)

# ===== 寫入函數 =====
def add_data(date_str, time_str, item, price, number, people, reimbursement):
    try:
        # 驗證日期和時間格式
        datetime.datetime.strptime(date_str, '%Y-%m-%d')
        datetime.datetime.strptime(time_str, '%H:%M')

        price = float(price)
        number = float(number)
        expense = price * number

        new_row = [date_str, time_str, item, price, number, expense, people, reimbursement]
        worksheet.append_row(new_row, value_input_option='USER_ENTERED')

        # 取得最新資料的前5筆
        all_data = worksheet.get_all_values()
        df = pd.DataFrame(all_data[1:], columns=all_data[0])
        preview = df.head().to_string(index=False)

        return f"✅ 資料已成功寫入 Google 工作表！\n\n目前前5筆資料：\n{preview}"
    except ValueError:
        return "❌ 日期或時間格式錯誤，請使用 YYYY-MM-DD 與 HH:MM 格式"
    except Exception as e:
        return f"❌ 發生錯誤：{e}"

# ===== Gradio 介面 =====
with gr.Blocks() as demo:
    gr.Markdown("## 📊 Google 試算表資料輸入")
    with gr.Row():
        date_in = gr.Textbox(label="日期 (YYYY-MM-DD)")
        time_in = gr.Textbox(label="時間 (HH:MM)")
    item_in = gr.Textbox(label="品項")
    with gr.Row():
        price_in = gr.Number(label="單價")
        number_in = gr.Number(label="數量")
    people_in = gr.Textbox(label="請款人")
    reimbursement_in = gr.Dropdown(choices=["是", "否"], label="是否核銷", value="否")
    output = gr.Textbox(label="結果", lines=12)

    submit_btn = gr.Button("寫入 Google 工作表")
    submit_btn.click(
        fn=add_data,
        inputs=[date_in, time_in, item_in, price_in, number_in, people_in, reimbursement_in],
        outputs=output
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d30cd4d0f525f5c624.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
